# Age prediction using street view images
Link: https://www.kaggle.com/code/gcdatkin/age-prediction-from-images-cnn-regression/notebook 

In [12]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path

from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import r2_score

In [13]:
directories = [
    Path('/workspace/workspace/ufo-prediction/image_data_NLD'),
    Path('/workspace/workspace/ufo-prediction/image_data_ESP'),  # Add your second directory path here
    Path('/workspace/workspace/ufo-prediction/image_data_FRA')   # Add your third directory path here
]

In [14]:
filepaths = pd.Series(
    [item for directory in directories for item in directory.glob('**/*.jpg')],
    name='Filepath'
).astype(str)

def extract_age_and_name(filepath):
    filename = os.path.basename(filepath)  # Get the filename from the filepath
    age_and_name = filename.split('.')[0]  # Split by dot and take the first part
    age = ''.join(filter(str.isdigit, age_and_name))[:4]  # Extract first 4 digits for age
    if age:  # Ensure age string is not empty
        age = int(age)
    else:  # Default age if no digits found
        age = 0
    return age

ages = pd.Series(filepaths.apply(lambda x: extract_age_and_name(x)), name='Age')

# Filter to include only ages above 1900
filtered_ages = ages[ages > 1900]

# Ensure we only work with filepaths that have a corresponding age above 1900
filtered_filepaths = filepaths[ages > 1900]

images = pd.concat([filtered_filepaths, filtered_ages], axis=1).sample(frac=1.0, random_state=1).reset_index(drop=True)

In [15]:
images

,Filepath,Age
0,/workspace/workspace/ufo-prediction/image_data...,2007
1,/workspace/workspace/ufo-prediction/image_data...,1980
2,/workspace/workspace/ufo-prediction/image_data...,1928
3,/workspace/workspace/ufo-prediction/image_data...,1972
4,/workspace/workspace/ufo-prediction/image_data...,1935
...,...,...
2500,/workspace/workspace/ufo-prediction/image_data...,2008
2501,/workspace/workspace/ufo-prediction/image_data...,1952
2502,/workspace/workspace/ufo-prediction/image_data...,1993
2503,/workspace/workspace/ufo-prediction/image_data...,1989


In [16]:
# Split into train and test set
train_df, test_df = train_test_split(images, train_size=0.7, shuffle=True, random_state=1)

In [17]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [18]:
train_df

,Filepath,Age
372,/workspace/workspace/ufo-prediction/image_data...,1950
137,/workspace/workspace/ufo-prediction/image_data...,1966
1804,/workspace/workspace/ufo-prediction/image_data...,1973
2208,/workspace/workspace/ufo-prediction/image_data...,1930
2248,/workspace/workspace/ufo-prediction/image_data...,1993
...,...,...
960,/workspace/workspace/ufo-prediction/image_data...,1910
905,/workspace/workspace/ufo-prediction/image_data...,1972
1096,/workspace/workspace/ufo-prediction/image_data...,1956
235,/workspace/workspace/ufo-prediction/image_data...,1977


In [19]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Age',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Age',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Age',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=False
)

Found 1403 validated image filenames.
Found 350 validated image filenames.
Found 752 validated image filenames.


# Training

In [20]:
inputs = tf.keras.Input(shape=(120, 120, 3))
x = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='linear')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='mse'
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=1000,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=15,
            restore_best_weights=True
        )
    ]
)

Epoch 1/1000
44/44 [==============================] - 2s 27ms/step - loss: 3845826.5000 - val_loss: 3734112.2500
Epoch 2/1000
44/44 [==============================] - 1s 22ms/step - loss: 2246745.0000 - val_loss: 79683.0078
Epoch 3/1000
44/44 [==============================] - 1s 21ms/step - loss: 119362.8828 - val_loss: 77852.2422
Epoch 4/1000
44/44 [==============================] - 1s 24ms/step - loss: 89099.9531 - val_loss: 75795.2656
Epoch 5/1000
44/44 [==============================] - 1s 23ms/step - loss: 87488.8125 - val_loss: 74825.0312
Epoch 6/1000
44/44 [==============================] - 1s 23ms/step - loss: 86447.9062 - val_loss: 73605.9375
Epoch 7/1000
44/44 [==============================] - 1s 23ms/step - loss: 85315.6641 - val_loss: 73201.8281
Epoch 8/1000
44/44 [==============================] - 1s 21ms/step - loss: 84297.8828 - val_loss: 71230.8125
Epoch 9/1000
44/44 [==============================] - 1s 23ms/step - loss: 82960.7734 - val_loss: 69954.5469
Epoch 10/100

# Results

In [21]:
predicted_ages = np.squeeze(model.predict(test_images))
true_ages = test_images.labels

rmse = np.sqrt(model.evaluate(test_images, verbose=0))
print("     Test RMSE: {:.5f}".format(rmse))

r2 = r2_score(true_ages, predicted_ages)
print("Test R^2 Score: {:.5f}".format(r2))

24/24 [==============================] - 0s 16ms/step
     Test RMSE: 30.20472
Test R^2 Score: -0.07674


In [22]:
null_rmse = np.sqrt(np.sum((true_ages - np.mean(true_ages))**2) / len(true_ages))
print("Null/Baseline Model Test RMSE: {:.5f}".format(null_rmse))

Null/Baseline Model Test RMSE: 29.10849
